In [1]:
# loading the files

import json 
with open("reproduce-cases.ipynb", mode = "r", encoding= "utf-8") as f:
    fileoutput = json.loads(f.read())
# fileoutput

with open("reproduce-cases-testing.ipynb", mode = "r", encoding= "utf-8") as f:
    fileoutput1 = json.loads(f.read())


In [2]:
# putting both the files in the format that i want
# in the first dict and second dict first is source code as a list of strings
# second is output as a list of strings
# thrid is what kind of output it is for example ['data', 'text/plain']
# fourth is output type


first_dict = dict()
second_dict = dict()
for cell_count,d in enumerate(fileoutput["cells"]):
    # print(d)
    if d["cell_type"] == "code":
        # print(d["cell_type"].keys())
        if d["outputs"] != []:
            for o in d["outputs"]:
                # print(o.keys())
                first_dict[cell_count] = (d["source"],o[list(o.keys())[0]][list(o[list(o.keys())[0]].keys())[0]], [list(o.keys())[0], list(o[list(o.keys())[0]].keys())[0]],d["outputs"][0]["output_type"])
        else:
            first_dict[cell_count] = (d["source"], [], [], "")
        # break


for cell_count, d in enumerate(fileoutput1["cells"]):
    if d["cell_type"] == "code":
        # print(d)
        if d["outputs"] != []:
            for o in d["outputs"]:
                # print(o)
                second_dict[cell_count] = (d["source"],o[list(o.keys())[0]][list(o[list(o.keys())[0]].keys())[0]], [list(o.keys())[0], list(o[list(o.keys())[0]].keys())[0]],d["outputs"][0]["output_type"])
                # second_dict[d["id"]] = (d["source"],o["data"]["text/plain"])
        else:
            second_dict[cell_count] = (d["source"], [], [],"")


In [3]:
second_dict[1]

(['import urllib\n',
  '\n',
  "local_fname = 'weather-buoy-data.txt.gz'\n",
  "urllib.request.urlretrieve('https://www.ndbc.noaa.gov/data/historical/stdmet/42040h2021.txt.gz', local_fname)\n",
  '\n',
  '# np.__version__\n',
  '# !python -V\n',
  '# iii\n',
  '# !python4 -V'],
 ["('weather-buoy-data.txt.gz', <http.client.HTTPMessage at 0x1077d5270>)"],
 ['data', 'text/plain'],
 'execute_result')

In [4]:
import ast
types = []
for du, d1 in zip(first_dict, second_dict):
    # print(first_dict[d][2] != [])
    if first_dict[du][3] == second_dict[d1][3] and first_dict[du][2] != []:
        try:
            s = second_dict[du][0]
            source = []
            source = [line for line in s if line.strip() != '']
            source = [s.replace('\\n', '\n') for s in source]
            s1 = []
            for line in source:
                if line.lstrip()[0] != "#":
                    s1.append(line)
            if s1[-1][0] != " ":
                s1[-1] = s1[-1]+'\n'
                s1 = s1 + ["variable_that_has_type = type(("+s1[-1].rstrip()+"))\n", "outputt = "+s1[-1].rstrip()+"\n"]
                s1 = "".join(s1)
                # print(s1)
                exec("".join(s1))
                # print(outputt)
                # break
                types.append(variable_that_has_type)
            else:
                exec("".join(s1))
                types.append("No output")
        except Exception as e:
            types.append(e)
            try:
                liter = ast.literal_eval(outputt)
                # print(liter)
            except Exception as ea:
                pass
                # print(ea)
    else:
        # print(second_dict[du][0])
        s = second_dict[du][0]
        source = []
        source = [line for line in s if line.strip() != '']
        source = [s.replace('\\n', '\n') for s in source]
        s1 = []
        for line in source:
            if line.lstrip()[0] != "#":
                s1.append(line)
        
        if (s1 != []) and (s1[-1][0] != " "):
            s1[-1] = s1[-1]+'\n'
        s1 = "".join(s1)
        # print(s1)
        exec("".join(s1))
        # print(outputt)
        # break
        types.append("No output")
                
types 

[tuple,
 'No output',
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 bool,
 tuple,
 tuple,
 'No output',
 int,
 list,
 'No output']

In [5]:
def compare_strings(s, s1,i,j):
    # s = first_dict[13][1][:i+1]
    # s1 = second_dict[13][1][:j+1]
    # s1 = [i.replace("\\n", "\n").replace("[", "").replace("]", "").replace("'", "").replace(",", "").strip() for i in s1]
    # s = [i.replace("\n", "").replace("[", "").replace("]", "").replace("'", "").replace(",", "").strip() for i in s]
    li = []
    for line in s:
        d = dict()
        le = 0
        # li.append(Counter(line.split()))
        for count, word in enumerate(line.split()):
            le = le + len(word)
            if word in d.keys():
                # d[word][0] += 1
                # d[word][1].add(count+1)
                d[word][1].append((le-len(word), le))
            else:
                # d[word] = [1, {count+1}]
                d[word] = [1,[(le-len(word),le)]]
            
            le = le+1
        li.append(d)
    # print(li)
    cw = set(li[0].keys())
    for i in li[1:]:
        cw = cw.intersection(set(i.keys()))
    mcw = dict()
    for i in li:
        for k in i:
            if k in mcw.keys():
                mcw[k][0] += i[k][0]
                mcw[k][1].append(i[k][1])
            else:
                mcw[k] = [i[k][0], [i[k][1]]]
    # print(mcw["Tropical"])
    for k in mcw:
        ol = mcw[k]
        nl = [ol[0]]
        tl = []
        for sublist in ol[1]:
            tl += sublist
        nl.append(tl)
        nl[1] = list(set(nl[1]))
        mcw[k] = nl
    import re
    cwd = dict()
    for word in cw:
        cwd[word] = mcw[word]
    whole_percentage = []
    for count, st in enumerate(s1):
        # print(st)
        percentage = 0
        for word in cwd:
            strr = r"" + word
            reg = r"\b" + strr + r"\b"
            indexes = re.search(reg, st)
            # print(word, indexes)
            if cwd[word] is not None and indexes is not None:
                if  (indexes.start(), indexes.end()) in cwd[word][1]:
                    percentage +=1
        whole_percentage.append(percentage/len(cwd))
    return sum(whole_percentage) / len(whole_percentage)
    

In [6]:
def compare_tuples(t1, t2):
    score = 0
    for i in range(len(t1)):
        if isinstance(t1[i], str) and isinstance(t2[i], str):
            if t1[i] == t2[i]:
                score += 1
        elif isinstance(t1[i], (list, tuple)) and isinstance(t2[i], (list, tuple)):
            score += compare_tuples(t1[i], t2[i])
        elif isinstance(t1[i], (int, float)) and isinstance(t2[i], (int, float)):
            if abs(t1[i] - t2[i]) < 0.0001:
                score += 1
    return score / len(t1)

In [7]:
pip install numpy


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install scikit-learn


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
from sklearn.metrics import mean_absolute_error


In [10]:
import re
res = []
res_dict = dict()
for du, du1, ty in zip(first_dict, second_dict, types):
    # print(first_dict[du][1], second_dict[du1][1])
    # try:
        if ty == "No output":
            res_dict[(du, du1)] = 1
        res = []
        if ty is tuple:
            y = " ".join(second_dict[du1][1])
            x = " ".join(first_dict[du][1])
            pattern = r'<.*?>'
            match = re.search(pattern, x)
            match1 = re.search(pattern, y)
            if match and match1:
                obj_str = match.group(0)
                obj_str1 = match1.group(0)
                # print(type(obj_str[1:-1]))
                x = x.replace(obj_str, str("'" + obj_str.split(" ")[0][1:]+ "'"))
                y = y.replace(obj_str1, str("'" + obj_str1.split(" ")[0][1:] + "'"))            
            first_output = ast.literal_eval(x)
            second_output = ast.literal_eval(y)
            # print(first_output, second_output)
            # if first_output == second_output:
            # print(first_output, second_output, "same")
            ret = compare_tuples(first_output,second_output)
            res_dict[(du, du1)] = ret
            # print(f"{ret} reproducabilty for this tuple {du},{du1}, {first_output}", )
            res.append(ret)
            # print(sum(res)/len(res), "reproducabilty rate for tuples")
        elif ty is int:
            ii = "".join(second_dict[du1][1])
            jj = "".join(first_dict[du][1])
            if int(ii) == int(jj):
                res_dict[(du, du1)] = 0
            else:
                res_dict[(du,du1)] = 1
        elif ty is list:
            s = first_dict[du][1]
            s1 = second_dict[du1][1]
            oi = ast.literal_eval(" ".join(s))
            oi2 = ast.literal_eval(" ".join(s1))
            list_types_s = []
            list_types_s1 = []
            # for i in oi:
            #     list_types_s.append(type(i))
            # for i in oi2:
            #     list_types_s1.append(type(i))
            list1_str_index = 0
            list2_str_index = 0
            for t in oi:
                if isinstance(t, str):
                    list1_str_index += 1
                else:
                    break
            for t in oi2:
                if isinstance(t, str):
                    list2_str_index +=1
                else:
                    break
            # print(list1_str_index, list2_str_index)
            res2 = compare_strings(oi[:list1_str_index], oi2[:list2_str_index], list1_str_index, list2_str_index)
            res_dict[(du, du1)] = res2
        elif ty is np.ndarray:
            arr2 = second_dict[du1][1]
            arr1 = first_dict[du][1]
            res_dict[(du, du1)] = compare_numpy_arrays(arr1, arr2)
        elif ty is bool:
            if first_dict[du][1] == second_dict[du1][1]:
                 res_dict[(du, du1)] = 1
            else:
                res_dict[(du, du1)] = 0

for d, ty in zip(res_dict, types):
    print(f"Cell Num: {d}\nReproducability Rate: {res_dict[d]},\nType : {ty}")
    print()

Cell Num: (1, 1)
Reproducability Rate: 1.0,
Type : <class 'tuple'>

Cell Num: (2, 2)
Reproducability Rate: 1,
Type : No output

Cell Num: (3, 3)
Reproducability Rate: invalid syntax (<string>, line 1),
Type : <class 'numpy.ndarray'>

Cell Num: (4, 4)
Reproducability Rate: invalid syntax (<string>, line 1),
Type : <class 'numpy.ndarray'>

Cell Num: (5, 5)
Reproducability Rate: invalid syntax (<string>, line 1),
Type : <class 'numpy.ndarray'>

Cell Num: (6, 6)
Reproducability Rate: invalid syntax (<string>, line 1),
Type : <class 'numpy.ndarray'>

Cell Num: (7, 7)
Reproducability Rate: 1,
Type : <class 'bool'>

Cell Num: (8, 8)
Reproducability Rate: 1.0,
Type : <class 'tuple'>

Cell Num: (10, 10)
Reproducability Rate: 1.0,
Type : <class 'tuple'>

Cell Num: (11, 11)
Reproducability Rate: 1,
Type : No output

Cell Num: (12, 12)
Reproducability Rate: 1,
Type : <class 'int'>

Cell Num: (13, 13)
Reproducability Rate: 0.8,
Type : <class 'list'>

Cell Num: (14, 14)
Reproducability Rate: 1,
Type

In [11]:
!pip install nbformat

## Adding new cell

In [12]:
import nbformat
from nbformat.v4 import new_code_cell, new_markdown_cell, new_output
from nbclient import NotebookClient
from nbformat.v4 import new_notebook, new_code_cell

# Load the notebook
with open('reproduce-cases-testing.ipynb') as f:
    nb = nbformat.read(f, as_version=4)

new_indexes = []
for i, cell in enumerate(nb.cells):
    if cell.get('outputs'):
        cell = nb.cells[i]
        source = cell.source.splitlines()
        source = [line+"\n" for line in source if line.strip() != '']
        source = [s.replace('\\n', '\n') for s in source]
        s1 = []
        for line in source:
            if line.lstrip()[0] != "#":
                s1.append(line)
        if s1[-1][0] != " ":
            new_cell = new_code_cell()
            new_cell.source = "type(("+s1[-1].rstrip()+"))\n"
            nb.cells.insert(i+1, new_cell)
            new_indexes.append(i+1)
print()
# Save the modified notebook
with open('new_notebook.ipynb', 'w') as f:
    nbformat.write(nb, f)


In [13]:
!jupyter nbconvert --execute --to notebook --inplace new_notebook.ipynb

[NbConvertApp] Converting notebook new_notebook.ipynb to notebook
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
[NbConvertApp] Writing 19701 bytes to new_notebook.ipynb


In [14]:
import json
with open("new_notebook.ipynb", "r") as f:
    nb1 = nbformat.read(f, as_version=4)




In [15]:
import json 
with open("reproduce-cases.ipynb", mode = "r", encoding= "utf-8") as f:
    fist = nbformat.read(f, as_version=4)

In [16]:
new_indexes

[2, 5, 7, 9, 11, 13, 15, 18, 21, 23]

In [17]:
types = []
testing_outputs = []
for i, ce in enumerate(nb1.cells):
    if i in new_indexes and ce["cell_type"] == "code":
        types.append(ce["outputs"][0]["data"]["text/plain"])
    elif ce["cell_type"] == "code" and ce.get("outputs"):
        testing_outputs.append(ce["outputs"][0]["data"]["text/plain"])

In [18]:
original_outputs = []

for i, ce in enumerate(fist.cells):
    if ce.get('outputs'):
        original_outputs.append((ce["outputs"][0]["data"]["text/plain"], i))


    

In [19]:
len(types) == len(original_outputs) == len(testing_outputs)

True

In [20]:
for i in original_outputs:
    print(i[1])

1
3
4
5
6
7
8
10
12
13


In [21]:
types

['tuple',
 'numpy.ndarray',
 'numpy.ndarray',
 'numpy.ndarray',
 'numpy.ndarray',
 'bool',
 'tuple',
 'tuple',
 'int',
 'list']

In [150]:
new_list = list()

def compare_numpy_arrays(arr1, arr2):
    d= {}
    try:
        if arr1[0][0:5] == "array" and arr2[0][0:5] == "array":
            for u in range(len(arr1)):
                arr1[u] = arr1[u].replace("...,", "").strip()
            un = "".join(arr1)
#             print(un)
            lst_arr = np.array(eval("np." + un))
            new_arr = np.delete(np.delete(lst_arr,3,axis =0), -4, axis =0)
            new_arr = np.delete(np.delete(new_arr,3, axis = 1), -4, axis=1)
            for u in range(len(arr2)):
                arr2[u] = arr2[u].replace("...,", "").strip()
            un = "".join(arr2)
            lst_arr1 = np.array(eval("np." + un))
            mae = mean_absolute_error(new_arr, lst_arr1)
            return f"{mae} mae value"

        else:
            return "Not an numpy array"
    except Exception as e:
        print(e)
        return e

In [151]:
import re
res_dict = dict()

for i, (ty, out1, out2) in enumerate(zip(types, original_outputs, testing_outputs)):
    if ty == "tuple":
        y = out1[0]
        x = out2
        pattern = r'<.*?>'
        match = re.search(pattern, x)
        match1 = re.search(pattern, y)
        if match and match1:
            obj_str = match.group(0)
            obj_str1 = match1.group(0)
            x = x.replace(obj_str, str("'" + obj_str.split(" ")[0][1:]+ "'"))
            y = y.replace(obj_str1, str("'" + obj_str1.split(" ")[0][1:] + "'"))            
        first_output = ast.literal_eval(x)
        second_output = ast.literal_eval(y)
        ret = compare_tuples(first_output,second_output)
        res_dict[(out1[1], out1[1])] = ret
    elif ty == "int":
        ii = out1[0]
        jj = out2
        if int(ii) == int(jj):
            res_dict[(out1[1], out1[1])] = 1
        else:
            res_dict[(out1[1],out1[1])] = 0 
    elif ty == "list":
        s = out1[0]
        s1 = out2
        oi =  ast.literal_eval(s)
        oi2 = ast.literal_eval(s1)
        list_types_s = []
        list_types_s1 = []
        list1_str_index = 0
        list2_str_index = 0
        for t in oi:
            if isinstance(t, str):
                list1_str_index += 1
            else:
                break
        for t in oi2:
            if isinstance(t, str):
                list2_str_index +=1
            else:
                break
#         print(list1_str_index, list2_str_index)
        res2 = compare_strings(oi[:list1_str_index], oi2[:list2_str_index], list1_str_index, list2_str_index)
        res_dict[(out1[1], out1[1])] = res2
    elif ty  == "numpy.ndarray":
        arr1 = out1[0].strip().split("\n")
        arr2 = out2.strip().split("\n")
#         print(arr1)
        res_dict[(out1[1], out1[1])] = compare_numpy_arrays(arr1, arr2)
#         break
    elif ty is bool:
        if first_dict[du][1] == second_dict[du1][1]:
             res_dict[(out1[1], out1[1])] = 0
        else:
            res_dict[(out1[1], out1[1])] = 1
        

In [152]:
res_dict

{(1, 1): 1.0,
 (3, 3): '2.6794444446901857e-06 mae value',
 (4, 4): '0.0 mae value',
 (5, 5): '0.0 mae value',
 (6, 6): '0.0 mae value',
 (8, 8): 1.0,
 (10, 10): 1.0,
 (12, 12): 0,
 (13, 13): 0.6444444444444444}

In [37]:
# original_outputs